In [1]:
!pip install -q -U transformers datasets bitsandbytes peft trl accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [37]:
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from google.colab import userdata

import torch
import os
import time

In [23]:
os.environ["HUGGINGFACE_TOKEN"] = userdata.get('HF_TOKEN')

In [5]:
from datasets import load_dataset
dataset = load_dataset("oz1115/korea_summary_thesis")

README.md:   0%|          | 0.00/405 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/211M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/179M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/171M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/355433 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output', '__index_level_0__'],
        num_rows: 355433
    })
})

In [7]:
dataset['train'][0]

{'input': '당신은 인공지능 비서입니다. 주어진 문장 리스트를 모두 활용하여 가장 정확한 단락을 생성하세요.',
 'instruction': "이 문장들은 임의의 순서로 섞여 있습니다. 모든 문장을 활용하여 원본 단락의 순서와 내용을 올바른 순서로 재구성하세요.\n#문장 리스트: ['따라서, 반도체 메모리 장치의 테스트 시간을 줄이기 위하여 현재 여러 가지 장치 및 방법들이 개발되고 있는 추세이다.', '그러나 반도체 기술의 고집적화되고 있는 추세에 따라 하나의 메모리 칩에 집적되는 메모리 셀의 수가 점점 많아지고 있고, 따라서 제조공정의 발달에도 불구하고 칩 내의 페일 셀의 존재 가능성은 상대적으로 커지고 있다.', '만일 이러한 페일 셀에 대하여 정확한 테스트가 이루어지지 않는다면, 반도체 메모리 장치로서의 신뢰성을 확보할 수 없게 된다.반도체 메모리 장치에 대한 테스트 동작 시 1개의 메모리 셀 단위로 테스트를 진행하는 경우, 고집적화된 반도체 메모리 장치의 테스트 시간이 오래 걸리게 되고 이는 비용증가의 원인이 된다.', '이를 위해서는 메모리 칩 상에 불량 메모리 셀이 존재하지 않아야 한다.', '일반적으로 DRAM과 같은 반도체 메모리 장치는 데이터의 리드(read) 및 라이트(write) 동작이 정확하게 이루어져야 한다.']",
 'output': '일반적으로 DRAM과 같은 반도체 메모리 장치는 데이터의 리드(read) 및 라이트(write) 동작이 정확하게 이루어져야 한다. 이를 위해서는 메모리 칩 상에 불량 메모리 셀이 존재하지 않아야 한다. 그러나 반도체 기술의 고집적화되고 있는 추세에 따라 하나의 메모리 칩에 집적되는 메모리 셀의 수가 점점 많아지고 있고, 따라서 제조공정의 발달에도 불구하고 칩 내의 페일 셀의 존재 가능성은 상대적으로 커지고 있다. 만일 이러한 페일 셀에 대하여 정확한 테스트가 이루어지지 않는다면, 반도체 메모리 장치로서의 신뢰성을 확보할 수 없게 된다.반도체 메모리 장치에 대한 테스트 동작 시 1개의 메모리 셀 

**Gemma 모델의 한국어 요약 테스트**

In [ ]:
BASE_MODEL = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [30]:
doc = dataset['train']['instruction'][0]

In [31]:
print(doc)

이 문장들은 임의의 순서로 섞여 있습니다. 모든 문장을 활용하여 원본 단락의 순서와 내용을 올바른 순서로 재구성하세요.
#문장 리스트: ['따라서, 반도체 메모리 장치의 테스트 시간을 줄이기 위하여 현재 여러 가지 장치 및 방법들이 개발되고 있는 추세이다.', '그러나 반도체 기술의 고집적화되고 있는 추세에 따라 하나의 메모리 칩에 집적되는 메모리 셀의 수가 점점 많아지고 있고, 따라서 제조공정의 발달에도 불구하고 칩 내의 페일 셀의 존재 가능성은 상대적으로 커지고 있다.', '만일 이러한 페일 셀에 대하여 정확한 테스트가 이루어지지 않는다면, 반도체 메모리 장치로서의 신뢰성을 확보할 수 없게 된다.반도체 메모리 장치에 대한 테스트 동작 시 1개의 메모리 셀 단위로 테스트를 진행하는 경우, 고집적화된 반도체 메모리 장치의 테스트 시간이 오래 걸리게 되고 이는 비용증가의 원인이 된다.', '이를 위해서는 메모리 칩 상에 불량 메모리 셀이 존재하지 않아야 한다.', '일반적으로 DRAM과 같은 반도체 메모리 장치는 데이터의 리드(read) 및 라이트(write) 동작이 정확하게 이루어져야 한다.']


In [32]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

Device set to use cuda:0


In [33]:
messages = [
    {
        "role": "user",
        "content": "다음 내용을 자연스럽고 문맥에 맞게 완성하세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [34]:
prompt

"<bos><start_of_turn>user\n다음 내용을 자연스럽고 문맥에 맞게 완성하세요:\n\n이 문장들은 임의의 순서로 섞여 있습니다. 모든 문장을 활용하여 원본 단락의 순서와 내용을 올바른 순서로 재구성하세요.\n#문장 리스트: ['따라서, 반도체 메모리 장치의 테스트 시간을 줄이기 위하여 현재 여러 가지 장치 및 방법들이 개발되고 있는 추세이다.', '그러나 반도체 기술의 고집적화되고 있는 추세에 따라 하나의 메모리 칩에 집적되는 메모리 셀의 수가 점점 많아지고 있고, 따라서 제조공정의 발달에도 불구하고 칩 내의 페일 셀의 존재 가능성은 상대적으로 커지고 있다.', '만일 이러한 페일 셀에 대하여 정확한 테스트가 이루어지지 않는다면, 반도체 메모리 장치로서의 신뢰성을 확보할 수 없게 된다.반도체 메모리 장치에 대한 테스트 동작 시 1개의 메모리 셀 단위로 테스트를 진행하는 경우, 고집적화된 반도체 메모리 장치의 테스트 시간이 오래 걸리게 되고 이는 비용증가의 원인이 된다.', '이를 위해서는 메모리 칩 상에 불량 메모리 셀이 존재하지 않아야 한다.', '일반적으로 DRAM과 같은 반도체 메모리 장치는 데이터의 리드(read) 및 라이트(write) 동작이 정확하게 이루어져야 한다.']<end_of_turn>\n<start_of_turn>model\n"

Gemma-it 추론

In [38]:
start_time = time.time()
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
end_time = time.time()

In [36]:
print(outputs[0]["generated_text"][len(prompt):])

따라서, 반도체 메모리 장치의 테스트 시간을 줄이기 위하여 현재 여러 가지 장치 및 방법들이 개발되고 있는 추세이다.

그러나 반도체 기술의 고집적화되고 있는 추세에 따라 하나의 메모리 칩에 집적되는 메모리 셀의 수가 점점 많아지고 있고, 따라서 제조공정의 발달에도 불구하고 칩 내의 페일 셀의 존재 가능성은 상대적으로 커지고 있다.

만일 이러한 페일 셀에 대하여 정확한 테스트가 이루어지지 않는다면, 반도체 메모리 장치로서의 신뢰성을 확보할 수 없게 된다.반도체 메모리 장치에 대한 테스트 동작 시 1개의 메모리 셀 단위로 테스트를 진행하는 경우, 고집적화된 반도체 메모리 장치의 테스트 시간이 오래 걸리게 되고 이는 비용증가의 원인이 된다.

이를 위해서는 메모리 칩 상에 불량 메모리 셀이 존재하지 않아야 한다.

일반적으로 DRAM과 같은 반도체 메모리 장치는 데이터의 리드(read) 및 라이트(write) 동작이 정확하게 이루어져야 한다.


In [40]:
print(f'추론속도: {end_time-start_time:.2f}초')

추론속도: 8.83초


Gemma-it 파인튜닝

In [8]:
def generate_prompts(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        messages = [
            {"role": "user",
             "content": "다음 내용을 자연스럽고 문맥에 맞게 완성하세요:\n\n {}".format(example['instruction'][i])},
            {"role": "assistant",
             "content": "{}".format(example['output'][i])}
        ]
        chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        output_texts.append(chat_message)

    return output_texts

In [9]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['instruction'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 내용을 자연스럽고 문맥에 맞게 완성하고, 글을 요약해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['instruction'][i], example['output'][i]))
    return prompt_list

In [10]:
# 실제 훈련시에는 아래 즐 삭제 후 주석 해제
train_data = dataset['train'].select(range(500))
#train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
다음 내용을 자연스럽고 문맥에 맞게 완성하고, 글을 요약해주세요:

이 문장들은 임의의 순서로 섞여 있습니다. 모든 문장을 활용하여 원본 단락의 순서와 내용을 올바른 순서로 재구성하세요.
#문장 리스트: ['따라서, 반도체 메모리 장치의 테스트 시간을 줄이기 위하여 현재 여러 가지 장치 및 방법들이 개발되고 있는 추세이다.', '그러나 반도체 기술의 고집적화되고 있는 추세에 따라 하나의 메모리 칩에 집적되는 메모리 셀의 수가 점점 많아지고 있고, 따라서 제조공정의 발달에도 불구하고 칩 내의 페일 셀의 존재 가능성은 상대적으로 커지고 있다.', '만일 이러한 페일 셀에 대하여 정확한 테스트가 이루어지지 않는다면, 반도체 메모리 장치로서의 신뢰성을 확보할 수 없게 된다.반도체 메모리 장치에 대한 테스트 동작 시 1개의 메모리 셀 단위로 테스트를 진행하는 경우, 고집적화된 반도체 메모리 장치의 테스트 시간이 오래 걸리게 되고 이는 비용증가의 원인이 된다.', '이를 위해서는 메모리 칩 상에 불량 메모리 셀이 존재하지 않아야 한다.', '일반적으로 DRAM과 같은 반도체 메모리 장치는 데이터의 리드(read) 및 라이트(write) 동작이 정확하게 이루어져야 한다.']<end_of_turn>
<start_of_turn>model
일반적으로 DRAM과 같은 반도체 메모리 장치는 데이터의 리드(read) 및 라이트(write) 동작이 정확하게 이루어져야 한다. 이를 위해서는 메모리 칩 상에 불량 메모리 셀이 존재하지 않아야 한다. 그러나 반도체 기술의 고집적화되고 있는 추세에 따라 하나의 메모리 칩에 집적되는 메모리 셀의 수가 점점 많아지고 있고, 따라서 제조공정의 발달에도 불구하고 칩 내의 페일 셀의 존재 가능성은 상대적으로 커지고 있다. 만일 이러한 페일 셀에 대하여 정확한 테스트가 이루어지지 않는다면, 반도체 메모리 장치로서의 신뢰성을 확보할 수 없게 된다.반도체 메모리 장치에 대한 테스트 동작 시 1개의 메모리 셀 단위

In [11]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [13]:
BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

# special tokens 추가가 필요한 경우 (예: pad token이 없을 때)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Trainer 실행

In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 1,
        #max_steps=3000,
        max_steps=100,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Applying formatting function to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Applying formatting function to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/500 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:
trainer.train()

Step,Training Loss
100,1.589300


TrainOutput(global_step=100, training_loss=1.5893096923828125, metrics={'train_runtime': 93.5259, 'train_samples_per_second': 4.277, 'train_steps_per_second': 1.069, 'total_flos': 3619193194266624.0, 'train_loss': 1.5893096923828125})

In [43]:
# ✅ 모델 평가 모드로 전환
model.eval()

# ✅ 테스트할 문장
doc = dataset['train']['instruction'][0]

# ✅ Chat 형식 프롬프트 생성 (Gemma는 반드시 이 포맷 사용)
messages = [
    {
        "role": "user",
        "content": f"다음 내용을 자연스럽고 문맥에 맞게 완성하세요:\n\n{doc}"
    }
]

# ✅ 프롬프트 생성
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# ✅ Tokenize 입력
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# ✅ 추론 시작 시간
start_time = time.time()

# ✅ 텍스트 생성
with torch.no_grad():
    output_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=512,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )

# ✅ 추론 종료 시간
end_time = time.time()

# ✅ 디코딩
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# ✅ 출력
print("📘 추론 결과:\n", generated_text)
print(f"⏱️ 추론 소요 시간: {end_time - start_time:.2f}초")

📘 추론 결과:
 user
다음 내용을 자연스럽고 문맥에 맞게 완성하세요:

이 문장들은 임의의 순서로 섞여 있습니다. 모든 문장을 활용하여 원본 단락의 순서와 내용을 올바른 순서로 재구성하세요.
#문장 리스트: ['따라서, 반도체 메모리 장치의 테스트 시간을 줄이기 위하여 현재 여러 가지 장치 및 방법들이 개발되고 있는 추세이다.', '그러나 반도체 기술의 고집적화되고 있는 추세에 따라 하나의 메모리 칩에 집적되는 메모리 셀의 수가 점점 많아지고 있고, 따라서 제조공정의 발달에도 불구하고 칩 내의 페일 셀의 존재 가능성은 상대적으로 커지고 있다.', '만일 이러한 페일 셀에 대하여 정확한 테스트가 이루어지지 않는다면, 반도체 메모리 장치로서의 신뢰성을 확보할 수 없게 된다.반도체 메모리 장치에 대한 테스트 동작 시 1개의 메모리 셀 단위로 테스트를 진행하는 경우, 고집적화된 반도체 메모리 장치의 테스트 시간이 오래 걸리게 되고 이는 비용증가의 원인이 된다.', '이를 위해서는 메모리 칩 상에 불량 메모리 셀이 존재하지 않아야 한다.', '일반적으로 DRAM과 같은 반도체 메모리 장치는 데이터의 리드(read) 및 라이트(write) 동작이 정확하게 이루어져야 한다.']
model
따라서, 반도체 메모리 장치의 테스트 시간을 줄이기 위하여 현재 여러 가지 장치 및 방법들이 개발되고 있는 추세이다.

그러나 반도체 기술의 고집적화되고 있는 추세에 따라 하나의 메모리 칩에 집적되는 메모리 셀의 수가 점점 많아지고 있고, 따라서 제조공정의 발달에도 불구하고 칩 내의 페일 셀의 존재 가능성은 상대적으로 커지고 있다.

만일 이러한 페일 셀에 대하여 정확한 테스트가 이루어지지 않는다면, 반도체 메모리 장치로서의 신뢰성을 확보할 수 없게 된다.반도체 메모리 장치에 대한 테스트 동작 시 1개의 메모리 셀 단위로 테스트를 진행하는 경우, 고집적화된 반도체 메모리 장치의 테스트 시간이 오래 걸리게 되고 이는 비용증가의 원인이 된다.

이를 위해서는 메모리 칩 상에 불량 메모리